# Importing libraries

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import tensorflow as tf
import torch as t
import keras
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import MinMaxScaler


## Importing data

In [2]:
stock_data = pd.read_csv('https://raw.githubusercontent.com/Varun2063/stock-price-prediction/main/ABT.csv')
stock_data.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
10093,2020-03-26,71.080002,75.970001,71.000000,75.809998,75.809998,10530500
10094,2020-03-27,77.500000,77.500000,73.610001,74.559998,74.559998,8360600
10095,2020-03-30,82.830002,84.400002,78.470001,79.339996,79.339996,47540200
10096,2020-03-31,81.430000,81.489998,78.559998,78.910004,78.910004,23792300
10097,2020-04-01,77.000000,77.379997,75.349998,76.570000,76.570000,12198700


# Data Preprocessing

In [3]:
stock_data.Date = pd.to_datetime(stock_data["Date"])

In [4]:
stock_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1980-03-17,0.0,0.512028,0.497999,0.501506,0.009856,7513300
1,1980-03-18,0.0,0.512028,0.494492,0.505013,0.009925,5303400
2,1980-03-19,0.0,0.515535,0.503260,0.512028,0.010062,2523300
3,1980-03-20,0.0,0.513781,0.499753,0.499753,0.009821,4654800
4,1980-03-21,0.0,0.506767,0.499753,0.505013,0.009925,1332800


In [5]:
print((stock_data.Date[len(stock_data)-1] - stock_data.Date[0]).days/365.25,"years")

40.04106776180698 years


we have **40 years** worth of data but we are only using **last 10 years** of data to use in the LSTM model.

In [6]:
tenyrs = stock_data['Date'].max() - pd.DateOffset(years=10)

stock_data_1 = stock_data[stock_data['Date'] >= tenyrs]

In [7]:
stock_data_1.head()

,Date,Open,High,Low,Close,Adj Close,Volume
7580,2010-04-01,25.333441,25.415007,25.227884,25.410208,17.999283,12903500
7581,2010-04-05,25.424603,25.549351,25.285460,25.395815,17.989082,11130400
7582,2010-04-06,25.367025,25.486977,25.227884,25.352633,17.958494,15102300
7583,2010-04-07,25.285460,25.395815,25.131924,25.189501,17.842943,20577100
7584,2010-04-08,25.232681,25.280663,25.045561,25.083944,17.768171,15111200


In [8]:
print("Distribution of OPEN values in STOCK DATA")
fig = px.histogram(stock_data_1 , x = "Open")
fig.show()

Distribution of OPEN values in STOCK DATA


In [9]:
print("Distribution of CLOSE values in STOCK DATA")
fig = px.histogram(stock_data_1, x = "Close")
fig.show()

Distribution of CLOSE values in STOCK DATA


We have noticed that the data doesnot follow a normal distribution. So we are going to transform it into a normal distribution with min-max transform.

In [10]:
print("data before transformation")
fig = px.line(stock_data_1, x=stock_data_1.columns[0], y=stock_data_1.columns[1:6])
fig.show()

data before transformation


**Scaling Data**

In [11]:
data_2 = stock_data_1[[ 'Open']]
scaler = MinMaxScaler(feature_range=(0,1))
data_2 = scaler.fit_transform(data_2)
data_2 = pd.DataFrame(data_2, columns = ['Open'], index = [i for i in range(7580, 7580+len(stock_data_1))])
data_2.insert(0, "Date", stock_data_1['Date'])

In [12]:
data_2.head()

,Date,Open
7580,2010-04-01,0.052131
7581,2010-04-05,0.053431
7582,2010-04-06,0.052610
7583,2010-04-07,0.051447
7584,2010-04-08,0.050694


In [13]:
fig = px.histogram(data_2, x = "Open")
fig.show()

In [14]:
print("data after transformation")
fig = px.line(data_2, x=data_2.columns[0], y=data_2.columns[1:6])
fig.show()

data after transformation


# Splitting the data

We are splitting the data into 3 parts  ( TRAINING, VALIDATION, TESTING )

In [15]:
len(data_2)

2518

In [16]:
data_3 = data_2.iloc[:,1:2]
data_3.head()

,Open
7580,0.052131
7581,0.053431
7582,0.052610
7583,0.051447
7584,0.050694


In [17]:
train = data_3[:round(len(data_3)*0.80)].values
test  = data_3[round(len(data_3)*0.80):].values

In [18]:
len(train)

2014

In [19]:
len(test)

504

In [20]:
X_train = []
y_train = []
for i in range(60, len(train)):
    X_train.append(train[i-60:i, 0])
    y_train.append(train[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [21]:
y_train.shape

(1954,)

In [22]:
X_test = []
y_test = []
for i in range(60, len(test)):
    X_test.append(test[i-60:i, 0])
    y_test.append(test[i, 0])

X_test, y_test = np.array(X_test), np.array(y_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [23]:
X_test.shape

(444, 60, 1)

In [24]:
y_test.shape

(444,)

# Modelling the lstm model


we first use the simple **vanilla lstm model** with a dropout layer

In [25]:
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Sequential

# Modelling the network

# Model 1 ( loss = 4.0271 )

In [26]:
model = keras.models.Sequential()
model.add(LSTM(50, input_shape = (X_train.shape[1],1)))
model.add(Dropout(0.1))
model.add(Dense(units = 1))
model.compile( optimizer = "adam", loss = "mean_squared_error")

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                10400     
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 10451 (40.82 KB)
Trainable params: 10451 (40.82 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
history = model.fit(X_train, y_train, epochs =50, batch_size = 32)

Epoch 1/50
62/62 [==============================] - 11s 10ms/step - loss: 0.0081
Epoch 2/50
62/62 [==============================] - 1s 8ms/step - loss: 6.1267e-04
Epoch 3/50
62/62 [==============================] - 1s 9ms/step - loss: 5.8358e-04
Epoch 4/50
62/62 [==============================] - 1s 9ms/step - loss: 4.7236e-04
Epoch 5/50
62/62 [==============================] - 1s 10ms/step - loss: 5.0486e-04
Epoch 6/50
62/62 [==============================] - 1s 9ms/step - loss: 4.4477e-04
Epoch 7/50
62/62 [==============================] - 1s 9ms/step - loss: 3.8889e-04
Epoch 8/50
62/62 [==============================] - 0s 8ms/step - loss: 4.2037e-04
Epoch 9/50
62/62 [==============================] - 1s 9ms/step - loss: 3.6001e-04
Epoch 10/50
62/62 [==============================] - 0s 5ms/step - loss: 3.4962e-04
Epoch 11/50
62/62 [==============================] - 0s 5ms/step - loss: 4.2304e-04
Epoch 12/50
62/62 [==============================] - 0s 5ms/step - loss: 3.3536e-04
Ep

In [29]:
y_test = np.reshape(y_test,(-1,1))
real_stock_price = scaler.inverse_transform(y_test)

In [30]:
pre_price = model.predict(X_test)
pre_price = scaler.inverse_transform(pre_price)
plot_df = pd.DataFrame(pre_price,columns = ["predicted"])
plot_df.insert(0, "real", real_stock_price)

14/14 [==============================] - 0s 2ms/step


In [31]:
fig = px.line(plot_df, y = plot_df.columns[:2])
fig.show()

In [32]:
mse(plot_df["real"], plot_df["predicted"])

3.1389841290381497

In [33]:
loss_value = history.history['loss']
epoch = range(1, len(loss_value)+1)
px.line(x = epoch, y = loss_value, labels = {'x':'epochs',"y":"loss_value"}, title = "training loss")

the MSE during training was **loss: 0.00017439** but the MSE during testing is 4.02711 as found. This shows that the model has been **overfitted severely**. now we will try to overcome this fitting by


*   reducing the number of epochs
*   implementing more drouput



# model 2 ( loss = 6.3589 )

In [34]:
model1 = keras.models.Sequential()
model1.add(LSTM(30, input_shape = (X_train.shape[1],1)))
model1.add(Dropout(0.2))
model1.add(Dense(units = 1))
model1.compile( optimizer = "adam", loss = "mean_squared_error")

In [35]:
history = model1.fit(X_train, y_train, epochs = 10, batch_size = 32)

Epoch 1/10
62/62 [==============================] - 3s 7ms/step - loss: 0.0146
Epoch 2/10
62/62 [==============================] - 0s 7ms/step - loss: 0.0018
Epoch 3/10
62/62 [==============================] - 0s 7ms/step - loss: 0.0016
Epoch 4/10
62/62 [==============================] - 0s 5ms/step - loss: 0.0011
Epoch 5/10
62/62 [==============================] - 0s 5ms/step - loss: 0.0012
Epoch 6/10
62/62 [==============================] - 0s 5ms/step - loss: 0.0010
Epoch 7/10
62/62 [==============================] - 0s 5ms/step - loss: 9.5055e-04
Epoch 8/10
62/62 [==============================] - 0s 5ms/step - loss: 8.8525e-04
Epoch 9/10
62/62 [==============================] - 0s 5ms/step - loss: 7.8921e-04
Epoch 10/10
62/62 [==============================] - 0s 5ms/step - loss: 8.3917e-04


In [36]:
predicted_1 = model1.predict(X_test)
plot_df["predicted_1"] = scaler.inverse_transform(predicted_1)
plot_df

14/14 [==============================] - 0s 3ms/step


,real,predicted,predicted_1
0,61.209999,61.282494,61.136673
1,60.459999,61.047230,60.924629
2,61.490002,60.702251,60.691616
3,60.630001,60.711048,60.480301
4,60.970001,60.587082,60.281643
...,...,...,...
439,71.080002,68.429031,70.260132
440,77.500000,68.138535,69.532730
441,82.830002,70.320663,69.222221
442,81.430000,74.443733,69.521194


In [37]:
fig = px.line(plot_df, y = plot_df.columns)
fig.show()

In [38]:
mse(plot_df["real"], plot_df["predicted_1"])

14.812490351431103

simplifying the model has made the output more deviant from the target output. so we are going to add a few more layers to test it out.

# Model 3 ( loss = 2.9866 )

In [39]:
model_3 = Sequential()
model_3.add(LSTM(50, return_sequences = True, input_shape = (X_train.shape[1],1)))
model_3.add(Dropout(0.1))
model_3.add(LSTM(50, return_sequences = True))
model_3.add(Dropout(0.2))
model_3.add(LSTM(30))
model_3.add(Dropout(0.2))
model_3.add(Dense(units = 1))
model_3.compile(optimizer = "adam", loss = "mean_squared_error")

In [40]:
history = model_3.fit(X_train, y_train, epochs = 10, batch_size = 32)

Epoch 1/10
62/62 [==============================] - 8s 14ms/step - loss: 0.0080
Epoch 2/10
62/62 [==============================] - 1s 14ms/step - loss: 0.0014
Epoch 3/10
62/62 [==============================] - 1s 15ms/step - loss: 0.0012
Epoch 4/10
62/62 [==============================] - 1s 10ms/step - loss: 0.0012
Epoch 5/10
62/62 [==============================] - 1s 11ms/step - loss: 0.0011
Epoch 6/10
62/62 [==============================] - 1s 11ms/step - loss: 0.0011
Epoch 7/10
62/62 [==============================] - 1s 11ms/step - loss: 0.0010
Epoch 8/10
62/62 [==============================] - 1s 11ms/step - loss: 0.0011
Epoch 9/10
62/62 [==============================] - 1s 10ms/step - loss: 9.6874e-04
Epoch 10/10
62/62 [==============================] - 1s 11ms/step - loss: 0.0010


In [41]:
predicted_2 = model.predict(X_test)
predicted_2 = scaler.inverse_transform(predicted_2)
plot_df["predicted_2"] = predicted_2

14/14 [==============================] - 0s 2ms/step


In [42]:
fig = px.line( y=plot_df['real'], labels={'x': 'date', 'y': 'price'}, title='predicted vs target')
fig.update_traces(line=dict(color='blue'), name='terget')
fig.add_scatter(y=plot_df['predicted_2'], mode='lines', line=dict(color='red'), name='predicted')
fig.show()

In [43]:
mse(plot_df['real'], plot_df['predicted_2'])

3.1389841290381497

# Model 4 ( loss = 19.3456 )

In [44]:
model_4 = Sequential()
model_4.add(LSTM(units=30,return_sequences=True,input_shape=(X_train.shape[1], 1)))
model_4.add(Dropout(0.1))
model_4.add(LSTM(units=20,return_sequences=True))
model_4.add(Dropout(0.1))
model_4.add(LSTM(units=10,return_sequences=True))
model_4.add(Dropout(0.1))
model_4.add(LSTM(units=10))
model_4.add(Dropout(0.1))
model_4.add(Dense(units=1))
model_4.compile(optimizer='adam',loss='mean_squared_error')

model_4.fit(X_train,y_train,epochs=10,batch_size=32)

Epoch 1/10
62/62 [==============================] - 9s 13ms/step - loss: 0.0131
Epoch 2/10
62/62 [==============================] - 1s 13ms/step - loss: 0.0019
Epoch 3/10
62/62 [==============================] - 1s 13ms/step - loss: 0.0018
Epoch 4/10
62/62 [==============================] - 1s 13ms/step - loss: 0.0016
Epoch 5/10
62/62 [==============================] - 1s 13ms/step - loss: 0.0015
Epoch 6/10
62/62 [==============================] - 1s 13ms/step - loss: 0.0014
Epoch 7/10
62/62 [==============================] - 1s 13ms/step - loss: 0.0014
Epoch 8/10
62/62 [==============================] - 1s 13ms/step - loss: 0.0012
Epoch 9/10
62/62 [==============================] - 1s 13ms/step - loss: 0.0013
Epoch 10/10
62/62 [==============================] - 1s 13ms/step - loss: 0.0011


In [45]:
predicted_3 = model_3.predict(X_test)
predicted_3 = scaler.inverse_transform(predicted_3)
plot_df["predicted_3"] = predicted_3

14/14 [==============================] - 1s 5ms/step


In [46]:
fig = px.line( y=plot_df['real'], labels={'x': 'date', 'y': 'price'}, title='predicted vs target')
fig.update_traces(line=dict(color='blue'), name='terget')
fig.add_scatter(y=plot_df['predicted_3'], mode='lines', line=dict(color='red'), name='predicted')
fig.show()

In [47]:
mse(plot_df['real'], plot_df['predicted_3'])

6.193427947574418

# Outcomes:



*   **Out of the 4 models developed we see that the "model 3" has the lowest loss value.**
*   **we see that the 4th model has only captured the general trend than giving the specific outputs.**

